In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/LLM_Nutriplan

# Bước 1: Cài đặt thư viện

In [ ]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()

# 1. Cài Unsloth phiên bản tối ưu cho Colab
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# 2. Cài các thư viện phụ trợ (Bỏ giới hạn phiên bản xformers để tránh lỗi build)
if major_version >= 8:
    # Dành cho GPU đời mới (A100, H100, RTX 30xx/40xx)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Dành cho GPU đời cũ (Tesla T4 trên Colab Free, V100)
    !pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

# Bước 2: Load Model và Chuẩn bị Dataset

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import json

# 1. Cấu hình Model
max_seq_length = 2048 # Có thể tăng lên 4096 nếu GPU mạnh
dtype = None # None để tự động chọn (float16 cho T4, bfloat16 cho Ampere+)
load_in_4bit = True # Dùng 4-bit để tiết kiệm VRAM (Quan trọng)

# 2. Load Qwen-2.5-7B-Instruct (Phiên bản Unsloth tối ưu)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 3. Kỹ thuật LoRA (Low-Rank Adaptation)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank càng cao càng học chi tiết nhưng tốn VRAM (16-64 là ổn)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# 4. Hàm Format Dữ liệu (QUAN TRỌNG NHẤT)
# Hàm này biến đổi JSON tool_calls thành chuỗi text để model học cách output JSON
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = []
    mapper = {"system": "system", "user": "user", "assistant": "assistant", "tool": "tool"}

    for convo in convos:
        text = ""
        for turn in convo:
            role = mapper.get(turn["role"], "user")
            content = turn.get("content", "")

            # Xử lý đặc biệt nếu role là assistant và có tool_calls
            if role == "assistant" and "tool_calls" in turn:
                # Convert list tool_calls thành JSON string
                # Đây là cái Model sẽ học để output ra
                content = json.dumps(turn["tool_calls"], ensure_ascii=False)

            # Áp dụng ChatML format của Qwen (<|im_start|>role...)
            text += f"<|im_start|>{role}\n{content}<|im_end|>\n"

        texts.append(text)
    return { "text" : texts, }

# 5. Load Dataset của bạn
# Hãy upload file nutriplan_finetune_mixed.jsonl lên Colab/Server trước
dataset = load_dataset("json", data_files="train.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

print("Ví dụ dữ liệu sau khi format:")
print(dataset[0]["text"])

# Bước 3: Tiến hành Training (Fine-tuning)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Set True nếu dataset lớn để train nhanh hơn
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Với 50 dòng, 60 steps là khoảng 5-10 epochs (tùy batch).
        # Bạn có thể tăng lên 100 nếu thấy model chưa học được.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

# Bắt đầu train
trainer_stats = trainer.train()

# Bước 4: Test thử Model ngay sau khi train

In [ ]:
# Test với Inference
FastLanguageModel.for_inference(model) # Bật chế độ inference (nhanh hơn 2x)

# Prompt test
messages = [
    {"role": "system", "content": "You are the NutriPlan AI Assistant.\n\nYour capabilities include:\n1.  **General Chat**: Friendly conversation.\n2.  **Web Search**: Use `web_search` for real-time info, news, or general knowledge NOT in your database.\n3.  **Knowledge Base**: Use `search_knowledge_base` to find specific nutrition facts, recipes, or meal data stored in our internal NutriPlan database.\n4.  **Plan Modification**: Use commands to modify the user's meal plan.\n\nGUIDELINE:\n- Prioritize `search_knowledge_base` over `web_search` for nutrition questions to ensure accuracy with our standards.\n- If `search_knowledge_base` returns no results, fall back to `web_search`. \n\nFORMATTING GUIDELINES:\n- Use **Markdown** to format your responses.\n- Use **Bold** for food names and key nutritional numbers (e.g., **150 kcal**).\n- Use lists (- or *) to present ingredients or nutrition facts clearly.\n- If comparing foods, you can use a Markdown table.\n- Keep the tone helpful and professional."},
    {"role": "user", "content": "Đổi món đầu tiên của bữa trưa nay sang Phở Bò giúp tôi."}
]

# Format prompt thủ công giống lúc train
input_text = ""
for turn in messages:
    input_text += f"<|im_start|>{turn['role']}\n{turn['content']}<|im_end|>\n"
input_text += "<|im_start|>assistant\n" # Mồi cho model trả lời

inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
response = tokenizer.batch_decode(outputs)

print("Kết quả model sinh ra:")
print(response[0].split("<|im_start|>assistant\n")[-1].replace("<|im_end|>", ""))

# 5. Lưu model sau khi fine-tune

In [ ]:
from huggingface_hub import login

# Thay bằng token của bạn
login(os.getenv('HF_TOKEN'))
# Đặt tên repo trên Hugging Face (ví dụ: ten-cua-ban/NutriPlan-Qwen-LoRA)
repo_name = "nermadie/NutriPlan-Qwen2.5-7B-LoRA"

local_save_directory = "lora_adapters"
# Lưu Local (để backup)
# model.save_pretrained(local_save_directory)
# tokenizer.save_pretrained(local_save_directory)

# Đẩy lên Hugging Face
model.push_to_hub(repo_name, save_directory=local_save_directory)
tokenizer.push_to_hub(repo_name, save_directory=local_save_directory)

print(f"Đã đẩy LoRA adapter lên: https://huggingface.co/{repo_name}")

# 6. Kiểm thử

## 6.1. RAG

In [ ]:
!pip install pymongo sentence-transformers duckduckgo-search openai

In [ ]:
# Category mapping (from FOOD_CATEGORIES)
CATEGORY_LABELS = {
    0: 'Dairy', 1: 'Eggs', 2: 'Fish', 3: 'Gluten', 4: 'Peanuts',
    5: 'Sesame', 6: 'Shellfish', 7: 'Soy', 8: 'Tree Nuts',
    9: 'Chocolate', 10: 'Cilantro', 11: 'Kale', 12: 'Mayonnaise',
    13: 'Mushrooms', 14: 'Mustard', 15: 'Olives', 16: 'Onions',
    17: 'Pickles', 18: 'Protein Powder', 19: 'Shakes & Smoothies', 20: 'Sugar',
    21: 'Blue Cheese', 22: 'Butter', 23: 'Cheese', 24: 'Cottage Cheese',
    25: 'Cream', 26: 'Goat Cheese', 27: 'Milk', 28: 'Whey Powder', 29: 'Yogurt',
    30: 'Red Meat', 31: 'Beef', 32: 'Lamb', 33: 'Pork & Bacon',
    34: 'Sausages and Luncheon Meats', 35: 'Poultry', 36: 'Chicken',
    37: 'Duck', 38: 'Turkey', 39: 'Cod', 40: 'Salmon', 41: 'Sardines',
    42: 'Tilapia', 43: 'Trout & Snapper', 44: 'Tuna', 45: 'Clams',
    46: 'Crab', 47: 'Lobster', 48: 'Mussels', 49: 'Oysters',
    50: 'Scallops', 51: 'Shrimp', 52: 'Squid', 53: 'Vegetables',
    54: 'Artichoke', 55: 'Arugula', 56: 'Asparagus', 57: 'Beets',
    58: 'Bell Peppers', 59: 'Broccoli', 60: 'Brussel Sprouts',
    61: 'Cabbage', 62: 'Carrots', 63: 'Cauliflower', 64: 'Celery',
    65: 'Chili Peppers', 66: 'Cucumber', 67: 'Eggplant', 68: 'Garlic',
    69: 'Lettuce', 70: 'Potatoes & Yams', 71: 'Radish', 72: 'Spinach',
    73: 'Squash', 74: 'Tomato', 75: 'Zucchini', 76: 'Fruit',
    77: 'Apple', 78: 'Avocado', 79: 'Banana', 80: 'Blueberries',
    81: 'Coconut', 82: 'Dates', 83: 'Grapes', 84: 'Kiwi',
    85: 'Lemon', 86: 'Lime', 87: 'Mango', 88: 'Melon',
    89: 'Orange', 90: 'Peaches & Plums', 91: 'Pineapple',
    92: 'Raisins', 93: 'Raspberries', 94: 'Strawberries',
    95: 'Edamame', 96: 'Soy Milk', 97: 'Soy Sauce', 98: 'Tempeh',
    99: 'Tofu', 100: 'Grains', 101: 'Barley', 102: 'Bread',
    103: 'Breakfast Cereals', 104: 'Corn', 105: 'Oats', 106: 'Pastas',
    107: 'Quinoa', 108: 'Rice', 109: 'Rye', 110: 'Wheat',
    111: 'Legumes', 112: 'Beans', 113: 'Chickpeas', 114: 'Hummus',
    115: 'Lentils', 116: 'Almonds', 117: 'Brazil Nuts', 118: 'Cashews',
    119: 'Hazelnuts', 120: 'Pecans', 121: 'Pistachios', 122: 'Walnuts',
    123: 'Fish Sauce', 124: 'Honey', 125: 'Ketchup', 126: 'Mayonnaise',
    127: 'Mustard', 128: 'Pickles', 129: 'Spices and Herbs',
    130: 'Sweets', 131: 'Soups, Sauces, and Gravies',
    132: 'Baked Products', 133: 'Beverages', 134: 'Fast Foods',
    135: 'Ethnic Foods', 136: 'Supplements'
}

In [ ]:
def mongodb_vector_search(query, k=5):
    """
    Vector search trong MongoDB Atlas

    Args:
        query: Text query (English hoặc Vietnamese)
        k: Số lượng kết quả trả về

    Returns:
        List of matching documents với scores
    """
    query_embedding = embedding_model.embed_query(query)

    # Aggregation pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",  # Tên index của bạn
                "path": "embedding",
                "queryVector": query_embedding,
                "numCandidates": 100,
                "limit": k
            }
        },
        {
            "$project": {
                "name": 1,
                "text_content": 1,
                "categories": 1,
                "nutrition.calories": 1,
                "nutrition.proteins": 1,
                "nutrition.carbs": 1,
                "nutrition.fats": 1,
                "nutrition.fiber": 1,
                "nutrition.sugar": 1,
                "nutrition.sodium": 1,
                "nutrition.cholesterol": 1,
                "nutrition.vitC": 1,
                "nutrition.calcium": 1,
                "nutrition.iron": 1,
                "nutrition.potassium": 1,
                "property.isBreakfast": 1,
                "property.isLunch": 1,
                "property.isDinner": 1,
                "property.isSnack": 1,
                "property.isDessert": 1,
                "property.mainDish": 1,
                "property.sideDish": 1,
                "property.totalTime": 1,
                "property.complexity": 1,
                "property.isHighProtein": 1,
                "property.isLowCarb": 1,
                "property.isLowFat": 1,
                "property.isHighFiber": 1,
                "property.majorIngredients": 1,
                "score": {"$meta": "vectorSearchScore"}
            }
        }
    ]

    # Giả sử collection đã được khai báo
    # from pymongo import MongoClient
    # client = MongoClient("your_connection_string")
    # db = client["your_database"]
    # collection = db["foods"]

    results = list(collection.aggregate(pipeline))
    return results

In [ ]:
def print_search_result(doc, index):
    """Helper function để in kết quả search (Clean for LLM)"""
    print(f"Document {index}:")
    print(f"Name: {doc.get('name', 'Unknown')}")
    print(f"Score: {doc.get('score', 0):.4f}")

    # Categories
    categories = doc.get('categories', [])
    if categories:
        cat_names = [CATEGORY_LABELS.get(c, str(c)) for c in categories]
        print(f"Categories: {', '.join(cat_names)}")

    # Nutrition
    nutr = doc.get('nutrition', {})
    nutr_parts = []
    for k, v in nutr.items():
        if v:
            nutr_parts.append(f"{k}: {v}")
    if nutr_parts:
        print(f"Nutrition: {', '.join(nutr_parts)}")

    # Properties
    prop = doc.get('property', {})
    props_list = []
    for key, val in prop.items():
        if isinstance(val, bool) and val:
            props_list.append(key)
        elif val and key != 'majorIngredients':
             props_list.append(f"{key}:{val}")

    if props_list:
         print(f"Properties: {', '.join(props_list)}")

    # Major ingredients
    major_ing = prop.get('majorIngredients', '')
    if major_ing:
        print(f"Ingredients: {major_ing}")

    # Text content preview
    text_content = doc.get('text_content', '')
    if text_content:
        print(f"Content: {text_content[:200]}...")
    print("-" * 20)

In [ ]:
import os
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer # Ví dụ dùng model open source
# import openai # Nếu bạn dùng OpenAI embedding

# --- 1. SETUP KẾT NỐI MONGODB ---
# Thay thế bằng URI thật của bạn
MONGO_URI = "os.getenv("MONGODB_URI")b.net/"
DB_NAME = "test"
COLLECTION_NAME = "foods"

try:
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]
    print("✅ Đã kết nối MongoDB thành công!")
except Exception as e:
    print(f"❌ Lỗi kết nối MongoDB: {e}")

# --- 2. SETUP EMBEDDING MODEL ---
# Bạn cần dùng đúng model mà bạn đã dùng để embed dữ liệu trong DB
class EmbeddingHandler:
    def __init__(self):
        # Ví dụ: Dùng model 'intfloat/multilingual-e5-small' (phổ biến, miễn phí)
        # Nếu bạn dùng OpenAI: self.client = openai.OpenAI(api_key="...")
        print("⏳ Đang load embedding model...")
        self.model = SentenceTransformer('intfloat/multilingual-e5-small')
        print("✅ Embedding model sẵn sàng!")

    def embed_query(self, text):
        # Nếu dùng SentenceTransformer:
        return self.model.encode(text).tolist()

        # Nếu dùng OpenAI:
        # response = self.client.embeddings.create(input=text, model="text-embedding-3-small")
        # return response.data[0].embedding

embedding_model = EmbeddingHandler()

# --- 3. HÀM CỦA BẠN (GIỮ NGUYÊN) ---
# (Paste hàm mongodb_vector_search và print_search_result của bạn vào đây)
# ... [Code của bạn ở trên] ...

# Hàm bổ sung: Chuyển kết quả RAG thành String để đưa vào Prompt cho AI
def format_rag_results_for_llm(results):
    context_str = ""
    for doc in results:
        name = doc.get('name', 'Unknown Dish')
        cal = doc.get('nutrition', {}).get('calories', 'N/A')
        protein = doc.get('nutrition', {}).get('proteins', 'N/A')
        text = doc.get('text_content', '')

        # Rút gọn text để tiết kiệm token
        summary = text[:300] + "..." if len(text) > 300 else text

        context_str += f"- Món: {name}\n  Dinh dưỡng: {cal} cal, {protein}g protein\n  Chi tiết: {summary}\n\n"
    return context_str

# --- 4. CHẠY TEST ---
if __name__ == "__main__":
    test_query = "Món ăn nào nhiều protein từ gà?"
    print(f"🔎 Đang tìm kiếm: '{test_query}'...")

    # Gọi hàm search
    results = mongodb_vector_search(test_query, k=3)

    if results:
        print(f"✅ Tìm thấy {len(results)} kết quả:")
        for i, doc in enumerate(results, 1):
            print_search_result(doc, i) # Hàm in đẹp của bạn
    else:
        print("❌ Không tìm thấy kết quả nào.")

## 6.2. DuckDuckGo

In [ ]:
from duckduckgo_search import DDGS

def web_search(query, max_results=3):
    """Tìm kiếm thông tin trên internet"""
    print(f"🌐 Đang Google search: {query}...")
    try:
        results = DDGS().text(keywords=query, max_results=max_results)
        context = ""
        for res in results:
            context += f"- Title: {res['title']}\n  Link: {res['href']}\n  Content: {res['body']}\n\n"
        return context
    except Exception as e:
        return f"Lỗi search web: {str(e)}"

## 6.3. Agent Controller

In [ ]:
agent_tools = [
    # --- NHÓM 1: INTERNAL TOOLS (AI dùng để tìm dữ liệu) ---
    {
        "type": "function",
        "function": {
            "name": "search_knowledge_base",
            "description": "Search internal database for nutrition info, recipes provided by NutriPlan.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Keywords regarding food/nutrition"}
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search internet for real-time info, prices, or external knowledge.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"}
                },
                "required": ["query"]
            }
        }
    },

    # --- NHÓM 2: FE CONTROL TOOLS (AI dùng để điều khiển App) ---
    {
        "type": "function",
        "function": {
            "name": "fe_swap",
            "description": "Swap two food items within the same meal or different meals.",
            "parameters": {
                "type": "object",
                "properties": {
                    "mealDate": {"type": "string", "description": "YYYY-MM-DD"},
                    "mealType": {"type": "string", "enum": ["breakfast", "lunch", "dinner", "snack"]},
                    "indexA": {"type": "integer"},
                    "indexB": {"type": "integer"}
                },
                "required": ["mealDate", "mealType", "indexA", "indexB"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "fe_reorder",
            "description": "Move an item to a new position within the same meal.",
            "parameters": {
                "type": "object",
                "properties": {
                    "mealDate": {"type": "string"},
                    "mealType": {"type": "string"},
                    "fromIndex": {"type": "integer"},
                    "toIndex": {"type": "integer"}
                },
                "required": ["mealDate", "mealType", "fromIndex", "toIndex"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "fe_move",
            "description": "Move an item from one meal to another.",
            "parameters": {
                "type": "object",
                "properties": {
                    "mealDate": {"type": "string"},
                    "fromMealType": {"type": "string"},
                    "fromIndex": {"type": "integer"},
                    "toMealType": {"type": "string"},
                    "toIndex": {"type": "integer"}
                },
                "required": ["mealDate", "fromMealType", "fromIndex", "toMealType", "toIndex"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "fe_replace",
            "description": "Replace a food item with a new one based on a query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "mealDate": {"type": "string"},
                    "mealType": {"type": "string"},
                    "targetIndex": {"type": "integer"},
                    "replacementQuery": {"type": "string", "description": "Name of the new food"},
                    "filters": {"type": "object", "description": "Optional filters like dishType, categoryIds"}
                },
                "required": ["mealDate", "mealType", "targetIndex", "replacementQuery"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "fe_apply_swap_option",
            "description": "Apply a specific pre-calculated swap option.",
            "parameters": {
                "type": "object",
                "properties": {
                    "optionIndex": {"type": "integer"}
                },
                "required": ["optionIndex"]
            }
        }
    }
]

In [ ]:
import json
from openai import OpenAI

# 1. Hàm map từ Tool Call sang JSON Client (Format cũ của bạn)
def map_to_client_response(tool_name, args):
    if tool_name == "fe_swap":
        return {"type": "swap", **args}
    elif tool_name == "fe_reorder":
        return {"type": "reorder", **args}
    elif tool_name == "fe_move":
        return {"type": "move", **args}
    elif tool_name == "fe_replace":
        return {
            "type": "replace_food",
            "mealDate": args.get("mealDate"),
            "mealType": args.get("mealType"),
            "targetIndex": args.get("targetIndex"),
            "replacementQuery": args.get("replacementQuery"),
            "limit": args.get("limit", 5),
            "filters": args.get("filters", {})
        }
    elif tool_name == "fe_apply_swap_option":
        return {"type": "apply_swap_option", **args}
    return None

# 2. Controller chính
def run_full_agent(user_query, chat_history=[]):
    client = OpenAI(base_url="http://localhost:8000/v1", api_key="EMPTY")

    messages = chat_history + [{"role": "user", "content": user_query}]

    # Cho phép AI suy nghĩ tối đa 3 bước (ví dụ: Search -> Search -> Replace)
    MAX_TURNS = 3

    for _ in range(MAX_TURNS):
        response = client.chat.completions.create(
            model="qwen-finetuned",
            messages=messages,
            tools=agent_tools,
            tool_choice="auto"
        )

        ai_msg = response.choices[0].message
        messages.append(ai_msg) # Cập nhật context

        # --- TRƯỜNG HỢP A: AI muốn gọi Tool ---
        if ai_msg.tool_calls:
            for tool_call in ai_msg.tool_calls:
                fn_name = tool_call.function.name
                fn_args = json.loads(tool_call.function.arguments)

                # 1. NHÓM INTERNAL (RAG / WEB) -> Xử lý và loop tiếp
                if fn_name == "search_knowledge_base":
                    # Gọi hàm mongodb_vector_search của bạn
                    print(f"🔄 Đang RAG: {fn_args['query']}")
                    rag_result = mongodb_vector_search(fn_args['query'])
                    content = format_rag_results_for_llm(rag_result) # Hàm format string

                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": content
                    })
                    continue # Quay lại đầu vòng for để AI đọc kết quả

                elif fn_name == "web_search":
                    # Gọi hàm search web
                    print(f"🌐 Đang Web Search: {fn_args['query']}")
                    web_result = web_search(fn_args['query'])

                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": web_result
                    })
                    continue # Quay lại đầu vòng for

                # 2. NHÓM TERMINAL (FE COMMANDS) -> Dừng và trả về Client
                # Nếu AI gọi tool này, nghĩa là nó đã quyết định hành động
                client_json = map_to_client_response(fn_name, fn_args)
                if client_json:
                    print(f"✅ Đã ra lệnh FE: {fn_name}")
                    # Có thể kèm theo message nếu muốn
                    # client_json["message"] = "Tôi đã thực hiện thay đổi..."
                    return client_json

        # --- TRƯỜNG HỢP B: AI trả lời Text (Chat thường) ---
        # Chỉ return khi AI không gọi tool nào nữa
        elif ai_msg.content:
            return {
                "type": "message",
                "text": ai_msg.content
            }

    return {"type": "message", "text": "Hệ thống đang xử lý quá nhiều bước."}

## 6.4. Test

In [ ]:
from unsloth import FastLanguageModel
import torch

# 1. Load Model Gốc + Adapter của bạn
# Unsloth thông minh sẽ tự biết load base model Qwen2.5-7B
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "nermadie/NutriPlan-Qwen2.5-7B-LoRA", # Trỏ thẳng vào repo LoRA của bạn
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# 2. Bật chế độ chạy nhanh (Inference)
FastLanguageModel.for_inference(model)

# 3. Test thử
messages = [
    {"role": "system", "content": "You are NutriPlan Assistant."},
    {"role": "user", "content": "Đổi món bữa trưa nay sang Phở bò."}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")

outputs = model.generate(inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0]))

In [ ]:
import json
import time

# Danh sách 10 Test Cases bao quát các chức năng
TEST_CASES = [
    # --- NHÓM 1: RAG & WEB SEARCH ---
    "Món 'Microwaved sweet potato' trong hệ thống có bao nhiêu calo và chế biến thế nào?",
    "Giá thịt lợn hôm nay tại Việt Nam bao nhiêu 1kg?",
    "Tôi đang tập gym, kiểm tra xem món 'Healthy Caesar Salad' có đủ protein không?",

    # --- NHÓM 2: FE CONTROL (BASIC) ---
    "Đổi chỗ món thứ 1 và món thứ 3 của bữa trưa nay.",
    "Đưa món canh ở bữa tối lên đầu tiên.",
    "Chuyển món tráng miệng (món cuối) của bữa trưa sang bữa phụ chiều.",

    # --- NHÓM 3: AGENTIC FLOW (SEARCH + ACTION) ---
    "Tìm công thức làm 'Ức gà sốt cam' trên mạng, nếu thấy ngon thì thay vào món thứ 1 bữa trưa nay.",
    "Thay món chính bữa tối nay bằng một món chay (vegan).",

    # --- NHÓM 4: KHÁC ---
    "Áp dụng lựa chọn thay thế số 2 đi.",
    "Replace the second item of breakfast with 'Tuna Salad'."
]

class Colors:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

def run_tests():
    print(f"{Colors.HEADER}{'='*60}")
    print(f"🚀 BẮT ĐẦU CHẠY 10 TEST CASES CHO NUTRIPLAN AGENT")
    print(f"{'='*60}{Colors.ENDC}\n")

    for i, question in enumerate(TEST_CASES, 1):
        print(f"{Colors.BOLD}▶ TEST {i}/{len(TEST_CASES)}:{Colors.ENDC} {question}")

        start_time = time.time()

        # --- GỌI AGENT ---
        # Giả lập lịch sử chat trống cho mỗi test case mới
        history = [{"role": "system", "content": "You are NutriPlan Assistant."}]

        try:
            # Sử dụng hàm run_full_agent đã định nghĩa ở cell trước
            result = run_full_agent(question, history)

            # --- XỬ LÝ KẾT QUẢ ĐỂ IN ĐẸP ---
            elapsed = time.time() - start_time

            # Phân loại màu sắc dựa trên kết quả
            color = Colors.GREEN
            result_type = result.get("type", "unknown")

            if result_type == "message":
                color = Colors.CYAN # Tin nhắn thường màu Cyan
            else:
                color = Colors.WARNING # Lệnh FE màu Vàng (Nổi bật)

            print(f"⏱️  Time: {elapsed:.2f}s")
            print(f"🤖 {Colors.BOLD}Output (JSON for FE):{Colors.ENDC}")

            # In JSON đẹp
            formatted_json = json.dumps(result, indent=2, ensure_ascii=False)
            # Tô màu từng dòng JSON để dễ nhìn
            print(f"{color}{formatted_json}{Colors.ENDC}")

        except Exception as e:
            print(f"{Colors.FAIL}❌ FAILED: {str(e)}{Colors.ENDC}")

        print(f"\n{Colors.BLUE}{'-'*60}{Colors.ENDC}\n")

if __name__ == "__main__":
    run_tests()